In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
%matplotlib inline

### Credit Card Example

In [2]:
cc_data = pd.read_csv('creditcard.csv')


In [3]:
cc_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
cc_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
X = cc_data.drop(['Class'], axis=1)
y = cc_data[['Class']]

### Neural Network (1000, 100, 10,)

In [7]:
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000, 100, 10))
mlp.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [8]:
mlp.score(X, y)

0.99827251436937992

In [9]:
from sklearn.model_selection import cross_val_score
mlp_results = cross_val_score(mlp, X, y, cv=5)
print(mlp_results)
print("5-fold cross validation average accuracy: %.3f" % (mlp_results.mean()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Ver

[ 0.998262    0.998262    0.99827952  0.99827952  0.99827952]
5-fold cross validation average accuracy: 0.998


### Random Forest

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
results = cross_val_score(rfc, X, y, cv=5)
print(results)
print("5-fold cross validation average accuracy: %.3f" % (results.mean()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Pytho

[ 0.06779959  0.99963133  0.99915732  0.99947332  0.99933288]
5-fold cross validation average accuracy: 0.813


### Commentary

With a large dataset such as this, with nearly 300,000 rows and 31 columns, the three-layered neural network predicts with a very high accuracy, possibly being overfit. It takes quite a time to run, however, compared the random forest. The random forest achieves good accuracy, probably has less danger of being overfit in this instance, but runs much more quickly. 

### Mutual Funds Example

In [3]:
vwob_data = pd.read_csv('Data/ETFs/vwob.us.txt')
veu_data = pd.read_csv('Data/ETFs/veu.us.txt')
vss_data = pd.read_csv('Data/ETFs/vss.us.txt')
vea_data = pd.read_csv('Data/ETFs/vea.us.txt')
vwo_data = pd.read_csv('Data/ETFs/vwo.us.txt')
vgk_data = pd.read_csv('Data/ETFs/vgk.us.txt')
vpl_data = pd.read_csv('Data/ETFs/vpl.us.txt')
vnqi_data = pd.read_csv('Data/ETFs/vnqi.us.txt')
#vigi_data = pd.read_csv('Data/ETFs/vigi.us.txt') -- 2/25/16 inception
#vymi_data = pd.read_csv('Data/ETFs/vymi.us.txt') -- 2/25/16 inception
bndx_data = pd.read_csv('Data/ETFs/bndx.us.txt')
vxus_data = pd.read_csv('Data/ETFs/vxus.us.txt')


In [4]:
vwob_data['symbol'] = 'vwob'
veu_data['symbol'] = 'veu'
vss_data['symbol'] = 'vss'
vea_data['symbol'] = 'vea'
vwo_data['symbol'] = 'vwo'
vgk_data['symbol'] = 'vgk'
vpl_data['symbol'] = 'vpl'
vnqi_data['symbol'] = 'vnqi'
bndx_data['symbol'] = 'bndx'
vxus_data['symbol'] = 'vxus'

In [5]:
etfs = [vwob_data, veu_data, vss_data, vea_data, vwo_data, vgk_data, vpl_data, vnqi_data, bndx_data, vxus_data]
etf_data = pd.concat(etfs).reset_index(drop=True)


In [8]:
etf_data['perc_gain'] = ((etf_data['Close'] - etf_data['Open'])/etf_data['Open'])*100
etf_data['max_range'] = abs(etf_data['High'] - etf_data['Low'])

In [9]:
etf_data.describe()

,Open,High,Low,Close,Volume,OpenInt,perc_gain,max_range
count,22730.000000,22730.000000,22730.000000,22730.000000,2.273000e+04,22730.0,22730.000000,22730.000000
mean,47.929206,48.140949,47.656703,47.916584,2.867061e+06,0.0,-0.021733,0.484246
std,16.545366,16.556426,16.523212,16.543340,6.209002e+06,0.0,0.940074,0.410320
min,15.858000,16.230000,15.671000,15.755000,5.880000e+02,0.0,-10.261642,0.000000
25%,37.170250,37.372250,36.950000,37.187750,1.369810e+05,0.0,-0.355977,0.251000
50%,44.743500,44.918000,44.490500,44.712000,5.269990e+05,0.0,0.000000,0.383000
75%,53.648000,53.806500,53.431750,53.624000,2.217943e+06,0.0,0.357745,0.591000
max,117.200000,117.570000,117.110000,117.570000,9.943128e+07,0.0,14.196252,12.447000


In [11]:
etf_data['gain'] = np.where(etf_data['perc_gain']>=0, '1', '0')

In [12]:
etf_data.head()

,Date,Open,High,Low,Close,Volume,OpenInt,symbol,perc_gain,max_range,gain
0,2013-06-07,64.661,64.661,63.965,64.269,55960,0,vwob,-0.606239,0.696,0
1,2013-06-10,64.451,64.451,63.367,63.640,98928,0,vwob,-1.258320,1.084,0
2,2013-06-11,63.234,63.234,62.378,62.615,40873,0,vwob,-0.978904,0.856,0
3,2013-06-12,64.095,64.095,63.432,63.432,8804,0,vwob,-1.034402,0.663,0
4,2013-06-13,63.836,64.040,63.557,64.040,132479,0,vwob,0.319569,0.483,1


In [13]:
X = etf_data[['Open', 'High', 'Low', 'Close', 'Volume']]
y = etf_data[['gain']]

### Neural Net

In [14]:
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000, 100, 10))
mlp.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [15]:
mlp.score(X, y)

0.5186537615486142

In [16]:
from sklearn.model_selection import cross_val_score
mlp_results = cross_val_score(mlp, X, y, cv=10)
print(mlp_results)
print("10-fold cross validation average accuracy: %.3f" % (mlp_results.mean()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Ver

[ 0.51846966  0.51869776  0.48130224  0.51869776  0.51869776  0.51869776
  0.51869776  0.48130224  0.51869776  0.51848592]
10-fold cross validation average accuracy: 0.511


In [18]:
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(500, 100, 50))
mlp.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 100, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [19]:
mlp.score(X, y)

0.48139023317201934

In [20]:
from sklearn.model_selection import cross_val_score
mlp_results = cross_val_score(mlp, X, y, cv=10)
print(mlp_results)
print("10-fold cross validation average accuracy: %.3f" % (mlp_results.mean()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Ver

[ 0.5180299   0.48130224  0.51869776  0.48130224  0.51869776  0.51869776
  0.48130224  0.48130224  0.51869776  0.48151408]
10-fold cross validation average accuracy: 0.500


In [21]:
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(5000, 500, 100, 10,))
mlp.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5000, 500, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [22]:
mlp.score(X, y)

0.48147822261328643

In [23]:
from sklearn.model_selection import cross_val_score
mlp_results = cross_val_score(mlp, X, y, cv=10)
print(mlp_results)
print("10-fold cross validation average accuracy: %.3f" % (mlp_results.mean()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Ver

[ 0.51846966  0.51869776  0.51869776  0.51869776  0.51869776  0.51693797
  0.51869776  0.51869776  0.53673559  0.51848592]
10-fold cross validation average accuracy: 0.520


### Random Forest

In [17]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
results = cross_val_score(rfc, X, y, cv=10)
print(results)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Pytho

[ 0.75549692  0.83106027  0.6458425   0.80730312  0.85305763  0.89309283
  0.79234492  0.85437747  0.74439067  0.78785211]
10-fold cross validation average accuracy: 0.796


### Commentary

For this smaller dataset of ETF prices, the random forest was a much better model. I attempted three different layering iterations of the neural network, but all three came out approximately the same -- about 50%. Additionally, the random forest had a much smaller runtime. 